## Load env file


In [7]:
from dotenv import load_dotenv

load_dotenv()


True

## initialize llm


In [8]:
from crewai import LLM

llm = LLM(model="gemini/gemini-2.0-flash", temperature=0.1)


## create a tool


In [ ]:
from crewai.tools import BaseTool

original_email = """
looping in Priya. TAS and PRX updates are in the deck. ETA for SDS integration is Friday.
Let's sync up tomorrow if SYNCBOT allows :-) . ping me if any blockers.
"""


class ReplacejargonsTool(BaseTool):
    name: str = "Replace Jargons"
    description: str = " Repl;ace Jargons with specific words"

    def _run(self, email: str) -> str:
        """
        This tool replaces jargons with specific words
        """

        replace_jargons = {
            " :-) ": "smily face",
            "ping me": "reach out to me",
            "wip": "work in progress",
            "ETA": "estimated time of arrival",
            "PRX": "product requirements exchange",
            "TAS": "technical assistance",
            "SDS": "software development standards",
            "SYNCBOT": "syncbot",
            "SYNCBOT": "syncbot",
            "SYNCBOT": "syncbot",
        }
        suggestions = []
        email_lower = email.lower()
        for jargon, replacement in replace_jargons.items():
            if jargon.lower() in email_lower:
                suggestions.append(f"consider replacing {jargon} -> {replacement}")
        return "\n".join(suggestions) if suggestions else "No jargons found"


jt = ReplacejargonsTool()
jt._run(original_email)

'consider replacing  :-)  -> smily face\nconsider replacing ping me -> reach out to me\nconsider replacing ETA -> estimated time of arrival\nconsider replacing PRX -> product requirements exchange\nconsider replacing TAS -> technical assistance\nconsider replacing SDS -> software development standards\nconsider replacing SYNCBOT -> syncbot'

## Create a Agent and Task


In [10]:
from crewai import Agent, Task, Crew

email_agent = Agent(
    role="Email Agent",
    goal="Send an email to the CEO of the company",
    backstory="You are a helpful assistant that can send emails to the CEO of the company",
    llm=llm,
    verbose=True,
    tools=[jt],
    # config = {
    #     "llm": llm,
    #     "tools": [send_email_tool],
    # }
)

original_email = """
hey team, just wanted to tell u that the demo is kind of ready, but there's still stuff left.
Maybe we can show what we have and say rest is WIP.
Let me know what u think. thanks
ping me if any blockers.

PRX and WIP are in the deck.
"""

email_task = Task(
    description=f"""Stake the following email and rewrite it to professional version. 
    Original email: '''{original_email}'''
    """,
    agent=email_agent,
    expected_output="An email to the CEO of the company, expand the aberrations",
)

crew = Crew(agents=[email_agent], tasks=[email_task], verbose=True)

result = crew.kickoff()

print(result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 2d19295b-bb74-4fd9-88b3-5d7d760bf1bd                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Email Agent                                                                                             │
│                                                                                                                 │
│  Task: Stake the following email and rewrite it to professional version.                                        │
│      Original email: '''                                                                                        │
│  hey team, just wanted to tell u that the demo is kind of ready, but there's still stuff left.                  │
│  Maybe we can show what we have and say rest is WIP.                                                            │
│  Let me know what u think. thanks                                                                               │
│  ping me if any blockers.                                                                                       │
│                                                                                                                 │
│  PRX and WIP are in the deck.                                                                                   │
│  '''                                                                                                            │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

/Users/jakeerchilakala/Documents/Projects/crewAgent/.venv/lib/python3.10/site-packages/rich/live.py:231: 
UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Email Agent                                                                                             │
│                                                                                                                 │
│  Thought: I need to rewrite the email to be more professional and suitable for the CEO. This involves           │
│  expanding abbreviations, using formal language, and providing a clear and concise message. I will use the      │
│  "Replace Jargons" tool to achieve this.                                                                        │
│                                                                                                                 │
│  Using Tool: Replace Jargons                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"email\": \"Subject: Demo Readiness and Next Steps\\n\\nDear CEO,\\n\\nI am writing to provide an update    │
│  on the demo. It is nearing completion, although some elements are still in progress.\\n\\nWe propose           │
│  showcasing the current version and indicating that the remaining features are Work In Progress.\\n\\nPlease    │
│  let me know your thoughts on this approach.\\n\\nThank you.\\n\\nKindly inform me if you encounter any         │
│  obstacles.\\n\\nPresentation and Work In Progress are in the deck.\\n\\nSincerely,\\n[Your Name]\"}"           │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  No jargons found                                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Email Agent                                                                                             │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Subject: Demo Readiness and Next Steps                                                                         │
│                                                                                                                 │
│  Dear CEO,                                                                                                      │
│                                                                                                                 │
│  I am writing to provide an update on the demo. It is nearing completion, although some elements are still in   │
│  progress.                                                                                                      │
│                                                                                                                 │
│  We propose showcasing the current version and indicating that the remaining features are Work In Progress.     │
│                                                                                                                 │
│  Please let me know your thoughts on this approach.                                                             │
│                                                                                                                 │
│  Thank you.                                                                                                     │
│                                                                                                                 │
│  Kindly inform me if you encounter any obstacles.                                                               │
│                                                                                                                 │
│  Presentation and Work In Progress are in the deck.                                                             │
│                                                                                                                 │
│  Sincerely,                                                                                                     │
│  [Your Name]                                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: b1b81b25-13db-4e75-b298-8e8c2a1719d0                                                                     │
│  Agent: Email Agent                                                                                             │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 2d19295b-bb74-4fd9-88b3-5d7d760bf1bd                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: Subject: Demo Readiness and Next Steps                                                           │
│                                                                                                                 │
│  Dear CEO,                                                                                                      │
│                                                                                                                 │
│  I am writing to provide an update on the demo. It is nearing completion, although some elements are still in   │
│  progress.                                                                                                      │
│                                                                                                                 │
│  We propose showcasing the current version and indicating that the remaining features are Work In Progress.     │
│                                                                                                                 │
│  Please let me know your thoughts on this approach.                                                             │
│                                                                                                                 │
│  Thank you.                                                                                                     │
│                                                                                                                 │
│  Kindly inform me if you encounter any obstacles.                                                               │
│                                                                                                                 │
│  Presentation and Work In Progress are in the deck.                                                             │
│                                                                                                                 │
│  Sincerely,                                                                                                     │
│  [Your Name]                                                                                                    │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Subject: Demo Readiness and Next Steps

Dear CEO,

I am writing to provide an update on the demo. It is nearing completion, although some elements are still in progress.

We propose showcasing the current version and indicating that the remaining features are Work In Progress.

Please let me know your thoughts on this approach.

Thank you.

Kindly inform me if you encounter any obstacles.

Presentation and Work In Progress are in the deck.

Sincerely,
[Your Name]
